In [1]:
import logging
import coloredlogs
import requests
from time import sleep

from psycopg2.extensions import cursor
from psycopg2.extensions import connection
import pandas as pd

In [2]:
logging.basicConfig()
# logger = logging.getLogger("poll-ooni")
# logger.setLevel(logging.DEBUG)
coloredlogs.install()
coloredlogs.install(level='DEBUG')
# coloredlogs.install(level='INFO')

In [3]:
import config
from imp import reload
reload(config)
from config import config
import psycopg2

# connect to the db
my_connection = psycopg2.connect(**config['postgres'])
my_cursor = my_connection.cursor()
logging.info('Connected to database.')

2021-06-01 12:52:59 congratulations root[20427] INFO Connected to database.


In [38]:
import src.ooni.utils as utils

<module 'src.ooni.utils' from '/home/ffff/Programs/taaraxtak/src/ooni/utils.py'>

In [12]:
maybe_t = utils.get_latest_reading_time(my_cursor)
maybe_t

datetime.datetime(2021, 5, 31, 10, 6, 56, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-420, name=None))

In [13]:
ms = utils.query_measurements_after(maybe_t)

2021-06-01 12:55:49 congratulations urllib3.connectionpool[20427] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-06-01 12:55:50 congratulations urllib3.connectionpool[20427] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-31T17:06:56 HTTP/1.1" 200 830785
2021-06-01 12:55:51 congratulations urllib3.connectionpool[20427] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-06-01 12:55:52 congratulations urllib3.connectionpool[20427] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-31T17%3A06%3A56&offset=1000 HTTP/1.1" 200 830728
2021-06-01 12:55:54 congratulations urllib3.connectionpool[20427] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-06-01 12:55:55 congratulations urllib3.connectionpool[20427] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?tes

2021-06-01 12:56:52 congratulations urllib3.connectionpool[20427] DEBUG Starting new HTTPS connection (1): api.ooni.io:443
2021-06-01 12:56:53 congratulations urllib3.connectionpool[20427] DEBUG https://api.ooni.io:443 "GET /api/v1/measurements?test_name=web_connectivity&anomaly=true&order_by=test_start_time&limit=1000&since=2021-05-31T17%3A06%3A56&offset=22000 HTTP/1.1" 200 189092


In [14]:
len(ms)

22227

In [19]:
for i, m in enumerate(ms):
    print(i, end=' ')
    utils.ingest_api_measurement(m)

2021-06-01 12:58:45 congratulations urllib3.connectionpool[20427] DEBUG Starting new HTTPS connection (1): publicsuffix.org:443
2021-06-01 12:58:46 congratulations urllib3.connectionpool[20427] DEBUG https://publicsuffix.org:443 "GET /list/public_suffix_list.dat HTTP/1.1" 200 None
2021-06-01 12:58:46 congratulations tldextract[20427] DEBUG computed TLD diff:
--- .tld_set_snapshot

+++ /usr/local/lib/python3.6/dist-packages/tldextract/.tld_set

@@ -5,24 +5,17 @@

 !city.sapporo.jp
 !city.sendai.jp
 !city.yokohama.jp
-!teledata.mz
 !www.ck
 *.bd
-*.bn
 *.ck
 *.er
-*.fj
 *.fk
-*.gu
 *.jm
 *.kawasaki.jp
-*.ke
 *.kh
 *.kitakyushu.jp
 *.kobe.jp
-*.kw
 *.mm
-*.mz
 *.nagoya.jp
 *.nom.br
 *.np
@@ -30,10 +23,7 @@

 *.sapporo.jp
 *.sch.uk
 *.sendai.jp
-*.ye
 *.yokohama.jp
-*.zm
-*.zw
 0.bg
 1.bg
 2.bg
@@ -45,6 +35,7 @@

 7.bg
 8.bg
 9.bg
+9guacu.br
 a.bg
 a.se
 aa.no
@@ -59,6 +50,7 @@

 abbott
 abbvie
 abc
+abc.br
 abeno.osaka.jp
 abiko.chiba.jp
 abira.hokkaido.jp
@@ -77,25 +69,29 @@

 ac.cn
 ac.

2021-06-01 12:58:46 congratulations tldextract[20427] WARNING unable to cache TLDs in file /usr/local/lib/python3.6/dist-packages/tldextract/.tld_set: [Errno 13] Permission denied: '/usr/local/lib/python3.6/dist-packages/tldextract/.tld_set'
2021-06-01 12:58:46 congratulations root[20427] DEBUG Time is in future: 2021-06-01 20:24:02+00:00. Setting time to now.
2021-06-01 12:58:46 congratulations root[20427] DEBUG Time is in future: 2021-06-01 20:17:10+00:00. Setting time to now.
2021-06-01 12:58:46 congratulations root[20427] DEBUG Time is in future: 2021-06-01 20:03:18+00:00. Setting time to now.


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

2021-06-01 12:58:59 congratulations root[20427] DEBUG Country code for .mobi is not a string: nan
2021-06-01 12:58:59 congratulations src.ooni.utils[20427] WARNING No TLD jurisidiction found for http://desiden.mobi/


1585 1586 1587 1588 1589 1590 1591 1592 1593 1594 1595 1596 1597 1598 1599 1600 1601 1602 1603 1604 1605 1606 1607 1608 1609 1610 1611 1612 1613 1614 1615 1616 1617 1618 1619 1620 1621 1622 1623 1624 1625 1626 1627 1628 1629 1630 1631 1632 1633 1634 1635 1636 1637 1638 1639 1640 1641 1642 1643 1644 1645 1646 1647 1648 1649 1650 1651 1652 1653 1654 1655 1656 1657 1658 1659 1660 1661 1662 1663 1664 1665 1666 1667 1668 1669 1670 1671 1672 1673 1674 1675 1676 1677 1678 1679 1680 1681 1682 1683 1684 1685 1686 1687 1688 1689 1690 1691 1692 1693 1694 1695 1696 1697 1698 1699 1700 1701 1702 1703 1704 1705 1706 1707 1708 1709 1710 1711 1712 1713 1714 1715 1716 1717 1718 1719 1720 1721 1722 1723 1724 1725 1726 1727 1728 1729 1730 1731 1732 1733 1734 1735 1736 1737 1738 1739 1740 1741 1742 1743 1744 1745 1746 1747 1748 1749 1750 1751 1752 1753 1754 1755 1756 1757 1758 1759 1760 1761 1762 1763 1764 1765 1766 1767 1768 1769 1770 1771 1772 1773 1774 1775 1776 1777 1778 1779 1780 1781 1782 1783 1784 

...

11242 11243 11244 11245 11246 11247 11248 11249 11250 11251 11252 11253 11254 11255 11256 11257 11258 11259 11260 11261 11262 11263 11264 11265 11266 11267 11268 11269 11270 11271 11272 11273 11274 11275 11276 11277 11278 11279 11280 11281 11282 11283 11284 11285 11286 11287 11288 11289 11290 11291 11292 11293 11294 11295 11296 11297 11298 11299 11300 11301 11302 11303 11304 11305 11306 11307 11308 11309 11310 11311 11312 11313 11314 11315 11316 11317 11318 11319 11320 11321 11322 11323 11324 11325 11326 11327 11328 11329 11330 11331 11332 11333 11334 11335 11336 11337 11338 11339 11340 11341 11342 11343 11344 11345 11346 11347 11348 11349 11350 11351 11352 11353 11354 11355 11356 11357 11358 11359 11360 11361 11362 11363 11364 11365 11366 11367 11368 11369 11370 11371 11372 11373 11374 11375 11376 11377 11378 11379 11380 11381 11382 11383 11384 11385 11386 11387 11388 11389 11390 11391 11392 11393 11394 11395 11396 11397 11398 11399 11400 11401 11402 11403 11404 11405 11406 11407 1140

12613 12614 12615 12616 12617 12618 12619 12620 12621 12622 12623 12624 12625 12626 12627 12628 12629 12630 12631 12632 12633 12634 12635 12636 12637 12638 12639 12640 12641 12642 12643 12644 12645 12646 12647 12648 12649 12650 12651 12652 12653 12654 12655 12656 12657 12658 12659 12660 12661 12662 12663 12664 12665 12666 12667 12668 12669 12670 12671 12672 12673 12674 12675 12676 12677 12678 12679 12680 12681 12682 12683 12684 12685 12686 12687 12688 12689 12690 12691 12692 12693 12694 12695 12696 12697 12698 12699 12700 12701 12702 12703 12704 12705 12706 12707 12708 12709 12710 12711 12712 12713 12714 12715 12716 12717 12718 12719 12720 12721 12722 12723 12724 12725 12726 12727 12728 12729 12730 12731 12732 12733 12734 12735 12736 12737 12738 12739 12740 12741 12742 12743 12744 12745 12746 12747 12748 12749 12750 12751 12752 12753 12754 12755 12756 12757 12758 12759 12760 12761 12762 12763 12764 12765 12766 12767 12768 12769 12770 12771 12772 12773 12774 12775 12776 12777 12778 1277

2021-06-01 13:00:44 congratulations root[20427] DEBUG Country code for .mobi is not a string: nan
2021-06-01 13:00:44 congratulations src.ooni.utils[20427] WARNING No TLD jurisidiction found for http://www.videolucah.mobi/


13801 13802 13803 13804 13805 13806 13807 13808 13809 13810 13811 13812 13813 13814 13815 13816 13817 13818 13819 13820 13821 13822 13823 13824 13825 13826 13827 13828 13829 13830 13831 13832 13833 13834 13835 13836 13837 13838 13839 13840 13841 13842 13843 13844 13845 13846 13847 13848 13849 13850 13851 13852 13853 13854 13855 13856 13857 13858 13859 13860 13861 13862 13863 13864 13865 13866 13867 13868 13869 13870 13871 13872 13873 13874 13875 13876 13877 13878 13879 13880 13881 13882 13883 13884 13885 13886 13887 13888 13889 13890 13891 13892 13893 13894 13895 13896 13897 13898 13899 13900 13901 13902 13903 13904 13905 13906 13907 13908 13909 13910 13911 13912 13913 13914 13915 13916 13917 13918 13919 13920 13921 13922 13923 13924 13925 13926 13927 13928 13929 13930 13931 13932 13933 13934 13935 13936 13937 13938 13939 13940 13941 13942 13943 13944 13945 13946 13947 13948 13949 13950 13951 13952 13953 13954 13955 13956 13957 13958 13959 13960 13961 13962 13963 13964 13965 13966 1396

InvalidCodepoint: Codepoint U+003A at position 4 of 'net:25461' not allowed

In [4]:
def rollback():
    my_cursor.execute("ROLLBACK")
    my_connection.commit()

In [41]:
geoip2.database.Reader?

In [40]:
import geoip2.database

# Visualization!

just connect to database to run cells below.

In [5]:
import json

def load (f):
    return json.load(open(f, 'r'))

sampleData = load('../../visualizations/globe/data/sampleData.json')

import country
export country
value

e -> i


countries are CC2s (perfect)

# Get directed data from the db

In [6]:
# rollback()

import pandas as pd

def unique_pairs (target_attribute:str, confirmed_only=True) -> pd.DataFrame:
    '''
    These are unique inputs that are blocked.
    
    We don't want every blocking example - if a site appears once or more, that's enough.
    '''
    my_cursor.execute(f'''
    SELECT DISTINCT input_url, probe_alpha2, {target_attribute} 
    from ooni_web_connectivity_test
    {"where confirmed=True" if confirmed_only else ""}
    ''')
    pairs = my_cursor.fetchall()
    return pd.DataFrame(pairs, columns=['input', 'e', 'i']).dropna()


In [18]:
tlds = unique_pairs('input_ip_alpha2')

In [19]:
tlds

,input,e,i
0,http://www.carnivalcasino.com/,RU,NL
1,http://www.proxyweb.net/,RU,DE
2,http://www.anonymsurfen.com/,SA,US
3,http://www.newnownext.com/,IR,NL
4,http://ilga.org/,IR,NL
...,...,...,...
1497,http://www.medinkur.ru/,RU,VG
1498,http://www.thegailygrind.com/,IR,US
1499,http://abc.go.com/,IR,DE
1500,http://khodorkovsky.ru/,RU,DE


In [52]:
def find_attacks (**kwargs) -> pd.DataFrame:
    # get data
    pairs_tld = unique_pairs('tld_jurisdiction_alpha2', **kwargs)
#     pairs_ip = unique_pairs('input_ip_alpha2', **kwargs)
    attacks = pairs_tld
#     attacks = pd.concat([pairs_tld, pairs_ip])
    return attacks

In [53]:
from typing import List

def find_unsafe_alpha2s (alpha2s: List[str]):
    '''
    Find alpha 2 country codes that are in our list, but not in the GIOJS sample dataset.
    '''
    cc2_in_their =  set(flatten([(s['e'], s['i']) for s in sampleData]))
    possibly_unsafe_cc2 = alpha2s.difference(cc2_in_their)
    return possibly_unsafe_cc2

In [54]:
def count_attacks (attacks_df: pd.DataFrame, from_alpha2: str, to_alpha2: str) -> int:
    return len(attacks_df[
        (attacks_df['e']==from_alpha2) & 
        (attacks_df['i']==to_alpha2)])

In [55]:
# rollback()
from funcy import flatten

def create_weights (magic_constant=100000, **kwargs) -> dict:
    '''
    magic_constant scales number of attacks for the viz.
    '''
    attacks = find_attacks(**kwargs)
    # find distinct lines of attack (unique attacker->attackee vectors) 
    lines_of_attack = set([(v[0], v[1]) for v in attacks[['e', 'i']].values])
    # find the CC2s that GioJS won't support
    observed_alpha2s =  set(flatten(attacks[['e', 'i']].values.tolist()))
    possibly_unsafe_cc2 = find_unsafe_alpha2s(observed_alpha2s)
    # construct weights
    weights = []
    for line in lines_of_attack:
        e, i = line
        if e not in possibly_unsafe_cc2 and i not in possibly_unsafe_cc2 and e != i:
            # count the number of attacks across each line
            num_attacks = count_attacks(attacks, e, i)
            weight = num_attacks*magic_constant
            weights.append({
                'e': e,
                'i': i,
                'v': weight
            })
    return weights

In [56]:
weights_confirmed = create_weights(confirmed_only=True)
weights_all = create_weights(confirmed_only=False)

In [57]:

weights = {
    'dataSetKeys': ['confirmed', 'all'],
    'initDataSetKey': 'confirmed',
    'confirmed': weights_confirmed,
    'all': weights_all
}

json.dump(weights, open('../../visualizations/globe/data/attacks.json', 'w'))

# Some basic questions

how many 'bullets' do we see going by?

In [58]:
pairs_tld = unique_pairs('tld_jurisdiction_alpha2', confirmed_only=True)
pairs_tld
# about 418?

,input,e,i
0,http://www.turkpress.co/,SA,CO
1,http://www.anonymsurfen.com/,SA,US
2,http://www.babylon-x.com/,SA,US
3,http://thenationalcampaign.org/,IR,US
4,http://www.ansar.org/,IR,US
...,...,...,...
716,http://www.centcom.mil/,IR,US
717,http://baskino.me/,RU,US
718,http://www.cbsnews.com/,IR,US
719,http://www.hustler.com/,IR,US


about 418/day?

by destination, who's receiving them?

In [38]:
pairs_ip.groupby('i').count().sort_values('e', ascending=False)

NameError: name 'pairs_ip' is not defined

maybe just a big datacenter there...? load balancer? unclear.
unclear what to make of input IP in general...